In [1]:
import rpt_parser
import pandas as pd

In [60]:
import pandas as pd


class OpenSTAParser:
    """

    Methodology:
    1. Read metadata on the type of file and run this is.
    2. Identify the boundary lines of the table data through regex matching.
    3. Read in table data of the timing performance.
    4. Understand the relevant timing and data parameters.
    5. Extract the relevant timing and data parameters from the
    """

    def __init__(
            self,
            file_address="25-rcx_sta.rpt",
    ):
        self.file_address = file_address

        # Internal
        self.file_lines_raw = None
        self.file_lines_raw = None
        self.file_lines_data = None
        self.propagation_delay = {}
        self.meta_data = pd.DataFrame()

        self.read_file_meta_data()
        self.extract_frame_meta_data()

        self.frame_timing_data = {}
        for frame_id in range(self.maximum_frame_amount):
            self.frame_timing_data[frame_id] = self.extract_timing_data(frame_id=frame_id)
            self.propagation_delay[frame_id] = self.calculate_propagation_delay(timing_data=self.frame_timing_data[frame_id])

    def read_file_meta_data(self):
        self.file = open(self.file_address, "r")
        self.file_lines_raw = self.file.readlines()
        self.file_lines_data = pd.DataFrame({"lines": self.file_lines_raw})
        self.configure_frame_id()
        self.configure_timing_data_rows()

    def configure_frame_id(self):
        self.file_lines_data["delimiters_line"] = self.file_lines_data.lines.str.contains("==========")
        self.file_lines_data["timing_data_line"] = self.file_lines_data.lines.str.contains("---------")
        frame_data = []
        frame_id_counter = 0
        parity_counter = 1
        row = 0
        for delimiter in self.file_lines_data["delimiters_line"]:
            if (delimiter):
                if (parity_counter % 2):
                    frame_id_counter += 1
                    parity_counter = 0
                else:
                    parity_counter += 1
            frame_data.append(frame_id_counter - 1)
            row += 1
        self.file_lines_data["frame_id"] = frame_data
        self.maximum_frame_amount = frame_id_counter

    def configure_timing_data_rows(self):
        self.frame_meta_data = {}
        for frame in range(self.maximum_frame_amount):
            timing_rows_index = self.file_lines_data.index[
                self.file_lines_data['timing_data_line'] & (self.file_lines_data['frame_id'] == frame)].tolist()
            if len(timing_rows_index) >= 1:

                self.frame_meta_data[frame] = {
                    "start_index": timing_rows_index[0],
                    "end_index": timing_rows_index[-1],
                    "start_rows_skip": timing_rows_index[0] + 1,
                    "end_rows_skip": len(self.file_lines_data) - timing_rows_index[-1],
                }
            else:
                self.frame_meta_data[frame] = {
                    "start_index": 0,
                    "end_index": 0,
                    "start_rows_skip": 0,
                    "end_rows_skip": 0,
                }
        return self.frame_meta_data

    def extract_frame_meta_data(self):
        # Extract Frame Metadata, note that these operations can be performed dataset wise.
        self.file_lines_data["start_point_line"] = self.file_lines_data.lines.str.contains("Startpoint")
        self.file_lines_data["end_point_line"] = self.file_lines_data.lines.str.contains("Endpoint")
        self.file_lines_data["path_group_line"] = self.file_lines_data.lines.str.contains("Path Group")
        self.file_lines_data["path_type_line"] = self.file_lines_data.lines.str.contains("Path Type")

        self.start_point_name = self.file_lines_data.lines[self.file_lines_data.start_point_line].str.extract(
            r'((?<=Startpoint:\s).*?(?=\s\())')
        self.end_point_name = self.file_lines_data.lines[self.file_lines_data.end_point_line].str.extract(
            r'((?<=Endpoint:\s).*?(?=\s\())')
        self.path_group_name = self.file_lines_data.lines[self.file_lines_data.path_group_line].str.extract(
            r'((?<=Path Group:\s).*)')
        self.path_type_name = self.file_lines_data.lines[self.file_lines_data.path_type_line].str.extract(
            r'((?<=Path Type:\s).*)')
        self.timing_data_start = self.file_lines_data.lines[self.file_lines_data.path_type_line].str.extract(
            r'((?<=Path Type:\s).*)')

    def extract_timing_data(self, frame_id=0):
        timing_data = pd.read_fwf(self.file_address,
                                  colspecs=[(0, 6), (6, 14), (14, 22), (22, 30), (30, 38), (38, 40), (40, 100)],
                                  skiprows=self.frame_meta_data[frame_id]["start_rows_skip"],
                                  skipfooter=self.frame_meta_data[frame_id]["end_rows_skip"],
                                  names=["Fanout", "Cap", "Slew", "Delay", "Time", "Direction", "Description"])
        timing_data["net"] = timing_data["Description"].str.extract(r'\(([^()]+)\)')
        return timing_data

    def calculate_propagation_delay(self,
                          net_in="in",
                          net_out="out",
                          timing_data=None,
                          ):
        return float(timing_data[timing_data.net == net_out].Time.values) - float(timing_data[timing_data.net == net_in].Time.values)


In [61]:
a = OpenSTAParser()
# a.calculate_propagation_delay(timing_data=a.frame_timing_data[0])
float(a.frame_timing_data[0][a.frame_timing_data[0].net == "out"].Time.values)

2.36

In [62]:
a.file_lines_data

,lines,delimiters_line,timing_data_line,frame_id,start_point_line,end_point_line,path_group_line,path_type_line
0,\n,False,False,-1,False,False,False,False
1,==============================================...,True,False,0,False,False,False,False
2,report_checks -unconstrained\n,False,False,0,False,False,False,False
3,==============================================...,True,False,0,False,False,False,False
4,Startpoint: in (input port clocked by __VIRTUA...,False,False,0,True,False,False,False
5,Endpoint: out (output port clocked by __VIRTUA...,False,False,0,False,True,False,False
6,Path Group: __VIRTUAL_CLK__\n,False,False,0,False,False,True,False
7,Path Type: max\n,False,False,0,False,False,False,True
8,\n,False,False,0,False,False,False,False
9,Fanout Cap Slew Delay Time Descr...,False,False,0,False,False,False,False


In [63]:
a.file_lines_data.lines[a.file_lines_data.start_point_line].str.extract(r'((?<=Startpoint:\s).*?(?=\s\())')
a.file_lines_data.lines[a.file_lines_data.end_point_line].str.extract(r'((?<=Endpoint:\s).*?(?=\s\())')
a.file_lines_data.lines[a.file_lines_data.path_group_line].str.extract(r'((?<=Path Group:\s).*)')
a.file_lines_data.lines[a.file_lines_data.path_type_line].str.extract(r'((?<=Path Type:\s).*)')

,0
7,max


In [69]:
import glob
import os


class RunAnalyser:
    """
    This class aims to list and perform analysis on all the relevant files in a particular run between all the corners.
    """
    def __init__(
            self,
            run_directory=None,
    ):
        self.run_directory = run_directory
        
        # Internal
        self.all_rpt_files_list = None
        self.timing_sta_files_list = None
        self.power_sta_files_list = None
        
        self.get_all_rpt_files()
    
    def get_all_rpt_files(self):
        self.all_rpt_files_list = []
        self.timing_sta_files_list = []
        self.power_sta_files_list = []
        PATH = os.path.dirname(os.getcwd())
        for x in os.walk(PATH):
            for file_path in glob.glob(os.path.join(x[0], '*.rpt')):
                self.all_rpt_files_list.append(file_path)
                if file_path.endswith(("sta.rpt", "sta.min.rpt", "sta.max.rpt", )):
                    self.timing_sta_files_list.append(file_path)
                if file_path.endswith(("power.rpt")):
                    self.power_sta_files_list.append(file_path)
    
    def extract_metrics_timing(self):
        """
        For every file in the sta timing file, extract the propagation delay and save the file meta data into a dicitonary.
        """
        self.timing_metrics_list = []
        for file in self.timing_sta_files_list:
            print(file)
            file_directory_data = os.path.normpath(file).split(os.path.sep)
            timing_data = OpenSTAParser(file_address=file)
            metrics = {
                "file_name": file_directory_data[-1],
                "flow_step_name": file_directory_data[-2],
                "propagation_delay": timing_data.propagation_delay,
            }
            self.timing_metrics_list.append(metrics)
        return self.timing_metrics_list
    
    def extract_metrics_power(self):
        pass

In [70]:
a = RunAnalyser()

In [71]:
a.extract_metrics_timing()

C:\Users\dario\Documents\rpt_parser\tests\25-rcx_sta.rpt
C:\Users\dario\Documents\rpt_parser\tests\src\inverter\runs\RUN_2023.03.30_15.10.55\reports\cts\10-cts_rsz_sta.max.rpt
C:\Users\dario\Documents\rpt_parser\tests\src\inverter\runs\RUN_2023.03.30_15.10.55\reports\cts\10-cts_rsz_sta.min.rpt
C:\Users\dario\Documents\rpt_parser\tests\src\inverter\runs\RUN_2023.03.30_15.10.55\reports\cts\10-cts_rsz_sta.rpt
C:\Users\dario\Documents\rpt_parser\tests\src\inverter\runs\RUN_2023.03.30_15.10.55\reports\placement\7-gpl_sta.max.rpt
C:\Users\dario\Documents\rpt_parser\tests\src\inverter\runs\RUN_2023.03.30_15.10.55\reports\placement\7-gpl_sta.min.rpt
C:\Users\dario\Documents\rpt_parser\tests\src\inverter\runs\RUN_2023.03.30_15.10.55\reports\placement\7-gpl_sta.rpt
C:\Users\dario\Documents\rpt_parser\tests\src\inverter\runs\RUN_2023.03.30_15.10.55\reports\placement\8-pl_rsz_sta.max.rpt
C:\Users\dario\Documents\rpt_parser\tests\src\inverter\runs\RUN_2023.03.30_15.10.55\reports\placement\8-pl_rsz_

[{'file_name': '25-rcx_sta.rpt',
  'flow_step_name': 'tests',
  'propagation_delay': {0: 0.3500000000000001, 1: 0.3500000000000001}},
 {'file_name': '10-cts_rsz_sta.max.rpt',
  'flow_step_name': 'cts',
  'propagation_delay': {0: 0.3400000000000003}},
 {'file_name': '10-cts_rsz_sta.min.rpt',
  'flow_step_name': 'cts',
  'propagation_delay': {0: 0.30000000000000027}},
 {'file_name': '10-cts_rsz_sta.rpt',
  'flow_step_name': 'cts',
  'propagation_delay': {0: 0.3400000000000003, 1: 0.3400000000000003}},
 {'file_name': '7-gpl_sta.max.rpt',
  'flow_step_name': 'placement',
  'propagation_delay': {}},
 {'file_name': '7-gpl_sta.min.rpt',
  'flow_step_name': 'placement',
  'propagation_delay': {}},
 {'file_name': '7-gpl_sta.rpt',
  'flow_step_name': 'placement',
  'propagation_delay': {}},
 {'file_name': '8-pl_rsz_sta.max.rpt',
  'flow_step_name': 'placement',
  'propagation_delay': {0: 0.3400000000000003}},
 {'file_name': '8-pl_rsz_sta.min.rpt',
  'flow_step_name': 'placement',
  'propagation_

In [18]:
os.path.normpath(a.power_sta_files_list[0]).split(os.path.sep)

['C:',
 'Users',
 'dario',
 'Documents',
 'rpt_parser',
 'tests',
 'src',
 'inverter',
 'runs',
 'RUN_2023.03.30_15.10.55',
 'reports',
 'cts',
 '10-cts_rsz_sta.power.rpt']